In [1]:
import numpy as np
from george import kernels

In [2]:
kernel = kernels.RotationKernel(
    log_amp=-2.1121901868039235, log_timescale=1.5, log_period=1.3322861105341484, log_factor=-5.0)

In [3]:
x = np.linspace(-10, 10, 500)
kernel.test_gradient(np.atleast_2d(x).T)

AssertionError: incorrect gradient for parameter 'log_amp' (0)

In [ ]:
import matplotlib.pyplot as pl

k = kernel.get_value(np.atleast_2d(x).T)
pl.figure(figsize=(6, 6))
pl.imshow(k, cmap="gray", interpolation="nearest")
pl.gca().set_xticklabels([])
pl.gca().set_yticklabels([]);

In [ ]:
log_amp: -2.1121901868039235
log_timescale: 1.5
log_period: 1.3322861105341484
log_factor: -5.0